In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm_notebook

from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import NearestNeighbors

from util import load_data, create_dataset

import time

In [7]:
(x_train, y_train), (x_test, y_test) = load_data(reshape=1)

In [10]:
start1 = time.time()
start2 = time.perf_counter()
start3 = time.process_time()



lof = LocalOutlierFactor(n_neighbors=20, novelty=True)
lof.fit(x_train[y_train==normal])

#Compute scores
scores = lof.decision_function(x_test)
# Baseline regularizaion and range normalization
scores = 1 - scores
scores = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))

#Compute labels and AUC
labels = np.copy(y_test)
labels[y_test == normal ] = 0
labels[y_test != normal ] = 1

AUC = roc_auc_score(labels, scores)


end1 = time.time()
end2 = time.perf_counter()
end3 = time.process_time()
print(end1 - start1)
print(end2 - start2)
print(end3 - start3)

146.39573788642883
146.3961792
145.59375


In [3]:
AUCs = []
for normal in range(10):
    lof = LocalOutlierFactor(n_neighbors=20, novelty=True)
    lof.fit(x_train[y_train==normal])
    
    #Compute scores
    scores = lof.decision_function(x_test)
    # Baseline regularizaion and range normalization
    scores = 1 - scores
    scores = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
    
    #Compute labels and AUC
    labels = np.copy(y_test)
    labels[y_test == normal ] = 0
    labels[y_test != normal ] = 1

    AUC = roc_auc_score(labels, scores)
    AUCs.append(AUC)
print(AUCs)

[0.9971910493687497, 0.9950320892685436, 0.9515153794023885, 0.9723099373341118, 0.9689853151809077, 0.9785450774069413, 0.9982631505306482, 0.9773093246433784, 0.916999305224105, 0.9763190125485026]
[0.9971910493687497, 0.9950320892685436, 0.9515153794023885, 0.9723099373341118, 0.9689853151809077, 0.9785450774069413, 0.9982631505306482, 0.9773093246433784, 0.916999305224105, 0.9763190125485026, 0.9971910493687497, 0.9950320892685436, 0.9515153794023885, 0.9723099373341118, 0.9689853151809077, 0.9785450774069413, 0.9982631505306482, 0.9773093246433784, 0.916999305224105, 0.9763190125485026, 0.9971910493687497, 0.9950320892685436, 0.9515153794023885, 0.9723099373341118, 0.9689853151809077, 0.9785450774069413, 0.9982631505306482, 0.9773093246433784, 0.916999305224105, 0.9763190125485026, 0.9971910493687497, 0.9950320892685436, 0.9515153794023885, 0.9723099373341118, 0.9689853151809077, 0.9785450774069413, 0.9982631505306482, 0.9773093246433784, 0.916999305224105, 0.9763190125485026, 0.9

# KNN

In [14]:
start = time.process_time()
knn = NearestNeighbors(n_neighbors=20).fit( x_train[y_train == normal] )

# Compute regular normal scores
distances, indices = knn.kneighbors( x_test )

end = time.process_time()
print(end - start)

91.84375


In [4]:
AUCs = []
for normal in range(10):
    knn = NearestNeighbors(n_neighbors=20).fit( x_train[y_train == normal] )
    
    # Compute regular normal scores
    distances, indices = knn.kneighbors( x_test )
    scores = np.mean(distances, axis=1)
    scores = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
    
    # Compute labels
    labels = np.copy(y_test)
    labels[y_test == normal ] = 0
    labels[y_test != normal ] = 1

    AUC = roc_auc_score(labels, scores)
    AUCs.append(AUC)
print(AUCs)

[0.9945002036291235, 0.9989909335082527, 0.9142817874406157, 0.931809271027214, 0.9412088924662555, 0.9459010557908979, 0.97520974953811, 0.9615299045365522, 0.8801544568258433, 0.9537383435632527]
[0.9945002036291235, 0.9989909335082527, 0.9142817874406157, 0.931809271027214, 0.9412088924662555, 0.9459010557908979, 0.97520974953811, 0.9615299045365522, 0.8801544568258433, 0.9537383435632527, 0.9945002036291235, 0.9989909335082527, 0.9142817874406157, 0.931809271027214, 0.9412088924662555, 0.9459010557908979, 0.97520974953811, 0.9615299045365522, 0.8801544568258433, 0.9537383435632527, 0.9945002036291235, 0.9989909335082527, 0.9142817874406157, 0.931809271027214, 0.9412088924662555, 0.9459010557908979, 0.97520974953811, 0.9615299045365522, 0.8801544568258433, 0.9537383435632527, 0.9945002036291235, 0.9989909335082527, 0.9142817874406157, 0.931809271027214, 0.9412088924662555, 0.9459010557908979, 0.97520974953811, 0.9615299045365522, 0.8801544568258433, 0.9537383435632527, 0.99450020362